This is a type of location optimization analysis, specifically finding the optimal location of facilites on a network. These are two types of Set-Coverage analysis implemented in **julia**:

### The Maximal Covering Location Problem (MCLP)
#### Objective: The Maximal Covering Location Problem determines the location of P facilities in order to maximize the demand covered within a pre-specified maximum distance coverage.

more information on GOSTNets Optimization can be found in the wiki: https://github.com/worldbank/GOST_PublicGoods/wiki/GOSTnets-Optimization

#### This is a Julia Notebook. If you are new to Julia, these are the [steps](https://datatofish.com/add-julia-to-jupyter/) to add Julia to a Jupyter Notebook

In [1]:
using Pkg
Pkg.add("JuMP")
Pkg.add("Cbc")
Pkg.add("MathOptInterface")
Pkg.add("MathProgBase")
Pkg.add("CSV")
Pkg.add("DelimitedFiles")
Pkg.add("DataFrames")
println("Done installing packages")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.to

In [2]:
#using JuMP, Cbc, GLPK, CPLEX, Test, Random, MathOptInterface, MathOptFormat, CSV, DataFrames, DelimitedFiles, MathProgBase
using JuMP, Cbc, MathOptInterface, CSV, DataFrames, DelimitedFiles, MathProgBase

In [3]:
# MathOptInterface is an abstraction layer for mathematical optimization solvers
const MOI = MathOptInterface

MathOptInterface

## This is the Julia Maximal Covering Location Problem (MCLP) function

In [101]:
function MaxCoveringProblem(CSVfile, pFacilities, maxCoverage::Int, origins_pop_dict)

    println("maxCoverage value")
    println(maxCoverage)

    # materialize a csv file as a DataFrame
    df = CSV.File(CSVfile) |> DataFrame!

    #extract column_headers
    column_headers = []
    #skip Column1
    for i=2:length(names(df))
      push!(column_headers,String(names(df)[i]))
    end
    
    OD_dict = Dict()
    for i in 1:size(df, 1)
        OD_dict[df[i,1]] = df[i,2:end]
    end

    #println("print OD_dict")
    #println(OD_dict)

    #origins as array
    origins = df[:,1]

    println("print origins length")
    print(length(origins))
    println("origins")
    println(origins)

    facilities = []
    for i in df[1,2:end]
      push!(facilities,trunc(Int, i))
    end

    println("facilities")
    println(facilities)

    #m = Model(with_optimizer(CPLEX.Optimizer))
    #output says threads were changed, but I do not see a difference on the resource monitor
    #m = Model(with_optimizer(Cbc.Optimizer, threads = 14))
    #change the limit to 
    m = Model(with_optimizer(Cbc.Optimizer, threads = 2, seconds = 68400))

    # Facility locations
    #@variable(m, 0 <= s[1:numLocation] <= 1)
    #@variable(m, 0 <= x[1:length(facilities)] <= 1)
    #binary variable
    @variable(m, x[1:length(facilities)], binary=true)

    #println("print Facility location var")
    #println(x)

    # Aux. variable: x_a,i = 1 if the closest facility to a is at i
    #@variable(m, 0 <= x[1:numLocation,1:numCustomer] <= 1)
    #@variable(m, 0 <= y[origins,1:length(facilities)] <= 1)
    #binary variable
    @variable(m, y[origins], binary=true)

    #println("print origin facility var")
    #println(y)

    # Objective: min distance
    #@objective(m, Min, sum(abs(customerLocations[a]-i)*x[i,a] for a = 1:numCustomer, i = 1:numLocation) )

    
    #println("testing1")
    #for j in facilities
    #    println(j)
    #end
    
    #@objective(m, Min, sum(OD_dict[i][j]*y[i,j] for i in origins, j = 1:length(facilities)) )
    
    if @isdefined origins_pop_dict
        println("origins_pop_dict exists")
        @objective(m, Max, sum(origins_pop_dict[i] * y[i] for i in origins))
        #@objective(m, Max, sum(y[i] for i in origins))
    else
        println("origins_pop_dict is not defined")
        @objective(m, Max, sum(y[i] for i in origins))
    end
    
    # Constraints


    # Subject to must allocate all facilities
    #@constraint(m, sum(x[i] for i=1:length(facilities)) == numFacility )


    for i in origins
        eligibleFacilities = []
        for j in 1:length(facilities)
            if OD_dict[i][j] <= maxCoverage
                push!(eligibleFacilities,j)
            end
        end
        @constraint(m, sum(x[j] for j in eligibleFacilities) >= y[i] )
    end

    @constraint(m, sum(x[j] for j in 1:length(facilities)) == pFacilities)

    JuMP.optimize!(m)

    println("Objective value is: ", JuMP.objective_value(m))

    #println("Objective bound is: ", JuMP.objective_bound(m))


    println("print array values")
    println(value.(x))
    println("print array length")
    println(length(value.(x)))

    result_array = value.(x)

    selected_facilities = []

    for i=1:length(result_array)
       if result_array[i] == 1
           push!(selected_facilities,column_headers[i])
       end
    end

    println("print selected_facilities")
    println(selected_facilities)



    if termination_status(m) == MOI.OPTIMAL
        optimal_solution = value.(x)
        optimal_objective = objective_value(m)
    elseif termination_status(m) == MOI.TIME_LIMIT && has_values(model)
        suboptimal_solution = value.(x)
        suboptimal_objective = objective_value(m)
    else
        error("The model was not solved correctly.")
    end

    return selected_facilities

end

MaxCoveringProblem (generic function with 1 method)

### The Max Covering function takes in the OD matrix as a csv file for the first argument, the number of P facilities as the second argument, and as the third argument it takes in a series of origins with their population.

### import an origins_pop_series from csv

In [85]:
# materialize a csv file as a DataFrame
origins_pop_series = CSV.File("../../../../lima_optimization_output/origins_w_demands_series_no_dupl.csv") |> DataFrame!

,NN,pop
,Int64,Float64
1,3,1458.0
2,21,2232.0
3,32,2041.0
4,82,1508.0
5,84,1610.0
6,99,1295.0
7,106,1216.0
8,114,824.0
9,124,440.0


In [86]:
typeof(origins_pop_series)

DataFrame

In [87]:
names(origins_pop_series)

2-element Array{Symbol,1}:
 :NN 
 :pop

In [88]:
#names!(origins_pop_series, [Symbol("col$i") for i in 1:2])
#names!(origins_pop_series,[Symbol("NN"),Symbol("pop")])

In [89]:
size(origins_pop_series)

(678, 2)

In [90]:
origins_pop_series

,NN,pop
,Int64,Float64
1,3,1458.0
2,21,2232.0
3,32,2041.0
4,82,1508.0
5,84,1610.0
6,99,1295.0
7,106,1216.0
8,114,824.0
9,124,440.0


In [91]:
origins_pop_series[1,2]

1458.0

In [92]:
origins_pop_dict = Dict()
for i in 1:size(origins_pop_series,1)
    origins_pop_dict[origins_pop_series[i,1]] = origins_pop_series[i,2]
end

In [93]:
origins_pop_dict

Dict{Any,Any} with 678 entries:
  3847 => 847.0
  1090 => 1371.0
  4130 => 130.0
  1333 => 141.0
  2812 => 1843.0
  3485 => 2131.0
  2564 => 1535.0
  5162 => 436.0
  5476 => 2520.0
  1662 => 1474.0
  1461 => 1715.0
  4223 => 1625.0
  1124 => 1705.0
  3181 => 1409.0
  6440 => 1525.0
  2835 => 2374.0
  1845 => 2921.0
  5784 => 2197.0
  563  => 4970.0
  2202 => 160.0
  3213 => 2394.0
  2354 => 2584.0
  671  => 110.0
  3126 => 4035.0
  5743 => 1392.0
  ⋮    => ⋮

In [94]:
origins_pop_dict[671]

110.0

In [105]:
selected_facilities = MaxCoveringProblem("../../../../lima_optimization_output/saved_OD.csv", 3, 200, origins_pop_dict)

maxCoverage value
200
print origins length
678origins
[6147, 2052, 3, 6154, 6162, 4115, 6165, 21, 4125, 32, 4130, 4132, 6181, 4134, 2087, 4139, 6190, 4144, 6193, 4146, 2099, 2100, 2096, 4151, 2103, 6204, 4157, 6207, 2112, 6214, 2119, 4167, 4170, 2125, 6222, 4176, 6224, 82, 84, 4180, 2135, 4183, 6233, 6234, 99, 6245, 2150, 6247, 6248, 4198, 106, 4204, 2157, 2160, 114, 2167, 2169, 4219, 124, 2175, 6271, 4223, 130, 6278, 6279, 4233, 4234, 6283, 6286, 6292, 6293, 150, 149, 4244, 2197, 152, 2202, 4252, 2196, 161, 163, 6307, 6311, 170, 173, 4269, 6318, 175, 2225, 174, 2231, 6332, 6334, 4302, 6355, 6357, 2263, 4312, 6361, 6363, 222, 224, 6369, 2278, 2280, 6377, 4329, 4340, 4341, 4342, 6393, 4348, 4354, 4356, 2319, 4373, 2328, 4378, 2334, 2335, 4388, 6437, 294, 295, 296, 297, 6440, 2343, 298, 4394, 2354, 2356, 4404, 4410, 321, 4419, 6478, 335, 6485, 6486, 2389, 2395, 6493, 2397, 351, 352, 6495, 6498, 4455, 4456, 2410, 2413, 368, 6513, 370, 6514, 4468, 2424, 380, 6528, 6529, 384, 4482, 386, 389

3-element Array{Any,1}:
 "2048"
 "3914"
 "474" 

In [106]:
selected_facilities

3-element Array{Any,1}:
 "2048"
 "3914"
 "474" 

In [45]:
#write-out selected_facilities
#writedlm("../../../../lima_optimization_output/selected_facilities_file_from_julia",selected_facilities)